### AGENTIC ROUTING

Unlike logical routing (pick ONE route), agentic routing:
- Agent dynamically decides which tools to use
- Can use multiple sources in a single query
- Most flexible, but usually slowest / most expensive

Key difference:
- Patterns 1–3: Static → pick ONE → execute
- Agentic: Dynamic → agent decides → may use MULTIPLE

Diagram:
    Query → Agent → [Tool 1] → [Tool 2] → ... → Synthesize → Response


In [1]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import MessagesPlaceholder, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.tools import tool
from pydantic import BaseModel, Field
from typing import Literal
import os
from sk import my_gpt
from config import (
    llm, embeddings, setup_vectorstores,
    get_retriever, format_docs
)


ImportError: cannot import name 'AgentExecutor' from 'langchain.agents' (D:\Anaconda\envs\rag\Lib\site-packages\langchain\agents\__init__.py)

In [2]:
def create_agent(vectorstores):
    """Create agent with multiple tools."""
    
    @tool
    def calculator(expression: str) -> str:
        """Calculate mathematical expressions."""
        try:
            return f"Result: {eval(expression)}"
        except Exception as e:
            return f"Error: {e}"
    
    @tool
    def search_cv(query: str) -> str:
        """Search CV/Resume for education, skills, work experience."""
        if "cv_resume" not in vectorstores:
            return "CV not available."
        docs = vectorstores["cv_resume"].similarity_search(query, k=3)
        return "\n\n".join([f"[CV] {d.page_content}" for d in docs])
    
    @tool
    def search_dms(query: str) -> str:
        """Search DMS (Deep Mutational Scanning) research documents."""
        if "dms_info" not in vectorstores:
            return "DMS docs not available."
        docs = vectorstores["dms_info"].similarity_search(query, k=3)
        return "\n\n".join([f"[DMS] {d.page_content}" for d in docs])
    
    @tool
    def search_llm_interview(query: str) -> str:
        """Search LLM/AI/ML interview preparation documents."""
        if "llm_interview" not in vectorstores:
            return "LLM Interview docs not available."
        docs = vectorstores["llm_interview"].similarity_search(query, k=3)
        return "\n\n".join([f"[LLM] {d.page_content}" for d in docs])
    
    @tool
    def search_all(query: str) -> str:
        """Search ALL documents at once."""
        results = []
        for name, vs in vectorstores.items():
            docs = vs.similarity_search(query, k=2)
            for d in docs:
                results.append(f"[{name}] {d.page_content[:300]}")
        return "\n\n---\n\n".join(results) if results else "No results."
    
    tools = [calculator, search_cv, search_dms, search_llm_interview, search_all]

    system = """You are an intelligent assistant with multiple tools.

    TOOLS:
    - calculator: Math calculations
    - search_cv: Search CV (education, skills, experience)
    - search_dms: Search DMS research
    - search_llm_interview: Search LLM interview prep
    - search_all: Search ALL documents
    
    STRATEGY:
    1. Analyze what information is needed
    2. Use appropriate tool(s) - you CAN use multiple
    3. Synthesize comprehensive answer
    4. Cite sources used

    Think step by step."""
    prompt = ChatPromptTemplate.from_messages([
        ("system", system),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ])
    
    agent = create_openai_tools_agent(llm, tools, prompt)
    return AgentExecutor(
        agent=agent, tools=tools, verbose=True,
        return_intermediate_steps=True, max_iterations=10
    )


In [3]:
def query_rag(question, vectorstores):
    print(f"\n{'='*80}")
    print(f" QUESTION: {question}")
    print(f"{'='*80}")
 
    
    agent = create_agent(vectorstores)
    result = agent.invoke({"input": question})
    
    # Extract tools used
    tools_used = []
    for step in result.get("intermediate_steps", []):
        action, _ = step
        tools_used.append(action.tool)
    
    print(f"\n TOOLS USED: {tools_used}")
    print(f"\n ANSWER:\n{'-'*80}\n{result['output']}\n{'-'*80}\n")
    
    return {"answer": result["output"], "tools_used": tools_used}

In [ ]:
vectorstores = setup_vectorstores()
    
test_questions = [
        "Where did Otabek study?",
        "What is the derivative of ReLU function?",
        "Compare Otabek's skills with LLM interview topics",
        "Calculate 5% of 10000 and tell me about DMS project",
        "Based on all documents, what should I study?",
    ]
    
for q in test_questions:
    query_rag(q, vectorstores)

In [ ]:
# =============================================================================
# AGENTIC ROUTER CLASS (Alternative Implementation)
# =============================================================================
class AgenticRouter:
    """
    Agentic Router with dynamic source selection.
    Analyzes query complexity and selects appropriate sources.
    """
    
    def __init__(self, vectorstores: Dict):
        self.vectorstores = vectorstores
        self.source_descriptions = SOURCE_DESCRIPTIONS
    
    def route_and_retrieve(self, query: str):
        """Agent analyzes query and decides which sources to use."""
        
        print(f"\n{'='*70}")
        print(" AGENTIC ROUTING")
        print(f"{'='*70}")
        print(f"Query: {query}\n")
        
        # Step 1: Analyze query complexity
        complexity_template = """Analyze this query:

    Query: {query}
    
    Is this a:
    1. SIMPLE query (needs one source)
    2. COMPLEX query (needs multiple sources)
    
    Type (SIMPLE/COMPLEX):"""

        prompt = ChatPromptTemplate.from_template(complexity_template)
        chain = prompt | llm | StrOutputParser()
        
        complexity = chain.invoke({"query": query}).strip().upper()
        print(f" Query complexity: {complexity}")
        
        # Step 2: Select sources based on complexity
        source_list = "\n".join([
            f"- {name}: {desc}"
            for name, desc in self.source_descriptions.items()
        ])
        
        if "SIMPLE" in complexity:
            # Single source selection
            select_template = """Select ONE best source:
    
    Sources:
    {sources}
    
    Query: {query}
    
    Best source (just the name):"""

            prompt = ChatPromptTemplate.from_template(select_template)
            chain = prompt | llm | StrOutputParser()
            
            selected = chain.invoke({
                "sources": source_list,
                "query": query
            }).strip().lower()
            
            sources_to_use = []
            for name in self.vectorstores.keys():
                if name in selected:
                    sources_to_use.append(name)
                    break
            
            if not sources_to_use:
                sources_to_use = [list(self.vectorstores.keys())[0]]
        
        else:  # COMPLEX - multiple sources
            select_template = """Select ALL relevant sources:

    Sources:
    {sources}
    
    Query: {query}
    
    List sources (comma-separated):"""

            prompt = ChatPromptTemplate.from_template(select_template)
            chain = prompt | llm | StrOutputParser()
            
            selected = chain.invoke({
                "sources": source_list,
                "query": query
            }).strip().lower()
            
            sources_to_use = []
            for name in self.vectorstores.keys():
                if name in selected:
                    sources_to_use.append(name)
            
            if not sources_to_use:
                sources_to_use = list(self.vectorstores.keys())
        
        print(f" Sources selected: {sources_to_use}")
        
        # Step 3: Query selected sources
        all_context = []
        for source in sources_to_use:
            if source in self.vectorstores:
                docs = self.vectorstores[source].similarity_search(query, k=3)
                if docs:
                    all_context.append(f"[{source.upper()}]\n" + format_docs(docs))
                    print(f"   ✓ {source}: {len(docs)} documents")
        
        # Step 4: Generate answer
        if not all_context:
            return sources_to_use, "No relevant information found."
        
        answer_template = """Answer based on the following sources:
    
    {context}
    
    Query: {query}
    
    Answer:"""

        prompt = ChatPromptTemplate.from_template(answer_template)
        chain = prompt | llm | StrOutputParser()
        
        answer = chain.invoke({
            "context": "\n\n".join(all_context),
            "query": query
        })
        
        print(f"\n{'='*70}")
        print(" ANSWER:")
        print(f"{'='*70}")
        print(answer)
        
        return sources_to_use, answer